In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install  datasets sacrebleu tensorboard -q


In [4]:
!pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.7 MB/s eta 0:00:00


# importing Libraries

In [5]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments, Trainer
from datasets import Dataset
from evaluate import load as load_metric
import pandas as pd
import torch


# Data preparation

In [6]:

data = []
with open("/kaggle/input/arabic-english-parallel-data/ara_.txt", "r", encoding="utf-8") as f:
    for line in f:
        if '\t' in line:
            en, ar = line.strip().split('\t')
            data.append({'translation': {'ar': ar, 'en': en}})

df = pd.DataFrame(data)
dataset = Dataset.from_pandas(df)


# Model Translation 

In [7]:
model_name = "Helsinki-NLP/opus-mt-ar-en"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/917k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.13M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

# Preprocessing

In [8]:
def preprocess(examples):
    inputs = [ex['ar'] for ex in examples['translation']]
    targets = [ex['en'] for ex in examples['translation']]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length").input_ids
    model_inputs["labels"] = labels
    return model_inputs

tokenized_dataset = dataset.map(preprocess, batched=True)


Map:   0%|          | 0/10742 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

# Training

In [9]:
training_args = TrainingArguments(
    output_dir="./results",
    logging_dir="./logs",
    logging_steps=100,
    per_device_train_batch_size=16,
    num_train_epochs=3,
    save_steps=500,
    save_total_limit=2,
    report_to="tensorboard",
    fp16=torch.cuda.is_available(),
)

In [10]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

trainer.train()


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss
100,0.608400
200,0.035700
300,0.033900
400,0.025400
500,0.023400
600,0.022000
700,0.020700
800,0.016400
900,0.016000
1000,0.017000


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[62833]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


TrainOutput(global_step=1008, training_loss=0.08138397770623367, metrics={'train_runtime': 466.464, 'train_samples_per_second': 69.086, 'train_steps_per_second': 2.161, 'total_flos': 1092408708169728.0, 'train_loss': 0.08138397770623367, 'epoch': 3.0})

In [11]:
model.save_pretrained("./my_finetuned_model")
tokenizer.save_pretrained("./my_finetuned_model")


('./my_finetuned_model/tokenizer_config.json',
 './my_finetuned_model/special_tokens_map.json',
 './my_finetuned_model/vocab.json',
 './my_finetuned_model/source.spm',
 './my_finetuned_model/target.spm',
 './my_finetuned_model/added_tokens.json')

# function for translation

In [12]:
def translate(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(model.device)
    output_ids = model.generate(**inputs, max_length=128, num_beams=4, early_stopping=True)
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)


In [14]:
for src, pred, ref in zip(srcs[:5], preds[:5], refs[:5]):
    print(f"Source: {src}")
    print(f"Predicted: {pred}")
    print(f"Reference: {ref}")
    print("="*50)


Source: مرحبًا.
Predicted: Hi.
Reference: Hi.
Source: اركض!
Predicted: Run!
Reference: Run!
Source: النجدة!
Predicted: Help!
Reference: Help!
Source: اقفز!
Predicted: Jump!
Reference: Jump!
Source: قف!
Predicted: Stop!
Reference: Stop!


# **Evaluation**

In [20]:
from evaluate import load as load_metric

metric = load_metric("sacrebleu")
sample = dataset.select(range(300)) 

srcs = [example['translation']['ar'] for example in sample]
refs = [example['translation']['en'] for example in sample]

preds = [translate(text) for text in srcs]

refs_lower = [ref.lower() for ref in refs]
preds_lower = [pred.lower() for pred in preds]

results = metric.compute(predictions=preds_lower, references=[[ref] for ref in refs_lower])
print("BLEU score:", results["score"])


BLEU score: 74.91640002191458


In [21]:
translate("البرمجة مفيدة")

'Programming is useful.'

In [22]:
translate("اخر سنة في الجامعة")

'Last year at the university.'

In [23]:
translate("ماذا تفعل الان؟")

'What are you doing now?'

In [25]:
translate("ما هو موضوع مشروع تخرجك؟")

"What's the subject of your graduate project?"